<a href="https://colab.research.google.com/github/Kkuntal990/CTW_2020/blob/master/siamese_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%tensorflow_version 1.x
import numpy as np # linear algebra
import tensorflow as tf
import numpy as np
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, LeakyReLU, Input, Conv2D, AveragePooling2D, MaxPooling2D, Dropout, Flatten, concatenate, Input, BatchNormalization, UpSampling2D
import tensorflow.keras.backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def get_data(data_file):
    f = h5py.File(data_file, 'r')
    H_Re = f['H_Re'][:] #shape (sample size, 56, 924, 5)
    H_Im = f['H_Im'][:] #shape (sample size, 56, 924, 5)
    SNR = f['SNR'][:] #shape (sample size, 56, 5)
    Pos = f['Pos'][:] #shape(sample size, 3)
    f.close()
    return H_Re, H_Im, SNR, Pos

def get_udata(data_file):
    f = h5py.File(data_file, 'r')
    print(f.keys())
    H_Re = f['H_Est'][:]
    f.close()
    return H_Re

def show_keys(data_file):
    f = h5py.File(data_file, 'r')
    for idx, key in enumerate(f.keys()):
      print(key, end=" ")

In [ ]:
H = get_udata('/content/drive/My Drive/CTW2020/Processed Data/udata2020.hdf5')

In [ ]:
from sklearn.model_selection import train_test_split

idx, idx_t = train_test_split(list(range(H.shape[0])), test_size=0.15, random_state=42, shuffle=True)
H= H[idx, :, :, :]

In [ ]:
print(H.shape)
H, H_v = train_test_split(H, test_size = 0.12, random_state = 54, shuffle = True)

In [ ]:
def Siamese_CC(inp_dim):
  input = Input(shape = (inp_dim))
  x = Conv2D(4,(3,9), padding='same',activation='relu', kernel_initializer='glorot_uniform')(input)
  x = BatchNormalization()(x)
  x = MaxPooling2D((1, 2), padding='same')(x)   
  x = Conv2D(8,(3,3), padding='same',activation='relu', kernel_initializer='glorot_uniform')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((1, 3), padding='same')(x)
  x = Conv2D(16,(2,4), padding='same',activation='relu', kernel_initializer='glorot_uniform')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((1, 1), padding='same')(x)
  x = Conv2D(32,(2,2), padding='same',activation='relu', kernel_initializer='glorot_uniform')(x)
  x = BatchNormalization()(x)
  encoded = MaxPooling2D((1,2), padding='same')(x)
  x = Conv2D(32,(2,2), padding='same',activation='relu', kernel_initializer='glorot_uniform')(encoded)
  x = BatchNormalization()(x)
  x = UpSampling2D((1,2))(x)
  x = Conv2D(16,(2,4), padding='same',activation='relu', kernel_initializer='glorot_uniform')(x)
  x = BatchNormalization()(x)
  x = UpSampling2D((1,1))(x)
  x = Conv2D(8,(2,4), padding='same',activation='relu', kernel_initializer='glorot_uniform')(x)
  x = BatchNormalization()(x)
  x = UpSampling2D((1, 3))(x)
  x = Conv2D(4,(3,3), padding='same',activation='relu', kernel_initializer='glorot_uniform')(x)
  x = BatchNormalization()(x)
  x = UpSampling2D((1, 2))(x)
  decoded = Conv2D(2,(3,9), padding='same',activation='relu', kernel_initializer='glorot_uniform')(x)
  return Model(input,encoded), Model(input, decoded)



def data_gen(data, BS):
    for i in range(0,len(data),BS):
        if i + BS > len(data)-1:
            yield (data[i:], data[i:])
        else:
            yield (data[i:i+BS],data[i:i+BS])

In [ ]:
Encoder, Decoder = Siamese_CC((56,924,2))
Decoder.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience = 30, verbose=1)
checkpointer = ModelCheckpoint('2/content/drive/My Drive/CTW2020/020_ae_test.h5', verbose=1, save_best_only=True)
Decoder.compile(optimizer='Adam', loss='mean_squared_error')

bs = 400   #batch size of the data
for i in range(200):
  Decoder.fit_generator(data_gen(H, bs), epochs=1,steps_per_epoch= len(H)//bs, verbose=1, validation_data= data_gen(H_v, bs), 
                      validation_steps = len(H_v)//bs, callbacks=[earlystopper, checkpointer],shuffle=True)

Instructions for updating:
Please use Model.fit, which supports generators.
31/31 [==============================] - ETA: 0s - loss: 0.1765
Epoch 00001: val_loss improved from inf to 0.04701, saving model to 2/content/drive/My Drive/CTW2020/020_ae_test.h5
31/31 [==============================] - ETA: 0s - loss: 0.0623
Epoch 00001: val_loss did not improve from 0.04701
31/31 [==============================] - ETA: 0s - loss: 0.0545
Epoch 00001: val_loss did not improve from 0.04701
31/31 [==============================] - ETA: 0s - loss: 0.0509
Epoch 00001: val_loss improved from 0.04701 to 0.04655, saving model to 2/content/drive/My Drive/CTW2020/020_ae_test.h5
31/31 [==============================] - ETA: 0s - loss: 0.0470
Epoch 00001: val_loss improved from 0.04655 to 0.04381, saving model to 2/content/drive/My Drive/CTW2020/020_ae_test.h5
31/31 [==============================] - ETA: 0s - loss: 0.0437
Epoch 00001: val_loss improved from 0.04381 to 0.04047, saving model to 2/content/

KeyboardInterrupt: ignored

In [ ]:
def Siamese_SS(inp_dim):
  inp1 = Input(shape=(inp_dim))
  inp2 = Input(shape=(inp_dim))

  enc1 = Encoder(inputs=inp1)
  enc2 = Encoder(inputs=inp2)

  x = Dense(2048, )    
  

  